In [1]:
#general 
import pandas as pd
import numpy as np
import os
from sklearn import __version__ as sklearn_version
from sklearn.model_selection import train_test_split

#warnings 
import warnings 
warnings.filterwarnings("ignore")

#for RNN
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import load_model


#for performance evaluation 
from sklearn.metrics import confusion_matrix
from pretty_confusion_matrix import pp_matrix
from sklearn.metrics import precision_recall_fscore_support

#for visualization 
import matplotlib.pyplot as plt
from wordcloud import WordCloud

import pickle

**Versions of Packages**

In [2]:
print("Pandas version:", pd.__version__)
print("Scikit-learn version:", sklearn_version)
print("NumPy version:", np.__version__)
print("TensorFlow version:", tf.__version__)
#unable to find the version for pretty_conf_matrix, but maybe only one of us runs it?

Pandas version: 1.4.4
Scikit-learn version: 1.2.2
NumPy version: 1.23.5
TensorFlow version: 2.10.0


In [3]:
#year_list = ['2009', '2010', '2011', '2012', '2013', '2014', '2015']
year_list = ['2016', '2017', '2018', '2019', '2020', '2021', '2022']
open_path = "C:/Users/danie/Desktop/Masters Thesis/New Clean Data for Log Reg/"
save_path = "C:/Users/danie/Desktop/Masters Thesis/RNN Models 2/"

In [4]:
def absolute_count(male_col, female_col):
    if female_col > male_col and male_col == 0:
        return 1
    elif male_col> female_col and female_col ==0: 
        return 0
    else: 
        return None

In [5]:
buffer_size = 50000 
batch_size = 64 # best practice
vocab_size = 60000 #this is 1/12 of all the words in the english language 


In [7]:
encoder = tf.keras.layers.TextVectorization(max_tokens=vocab_size)

In [8]:
#newmodel - BEST ONE SO FAR!!
rnn2 = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 256, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2)
])

In [9]:
for year in year_list:

    df = pd.read_pickle(open_path + year + "_final_RNN.pickle")

    #apply function to only get rows with an absolute count 
    df['col_type'] = df.apply(lambda row: absolute_count(row['male_count'], row['female_count']),axis=1)

    #remove nulls 
    df = df[df["col_type"].notnull()]

    # Split the df by the preprocessed 
    x_train, x_test, y_train, y_test = train_test_split(df["pre_processed_sent"], 
                                                        df["col_type"], 
                                                        stratify = df["col_type"])

    xTrain, xTest, yTrain, yTest = train_test_split(df["string_rnn"],
                                                    df["col_type"], 
                                                    stratify = df["col_type"])

    train_dataset = tf.data.Dataset.from_tensor_slices((xTrain, yTrain)) #string_rnn here 
    test_dataset = tf.data.Dataset.from_tensor_slices((xTest, yTest)) #clean text 

    train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


    encoder.adapt(train_dataset.map(lambda text, label: text))

    encoded_vocab = np.array(encoder.get_vocabulary())
    vocab_dict = dict(enumerate(encoded_vocab))

    rnn2.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                optimizer=tf.keras.optimizers.Adam(1e-4),
                metrics=['accuracy'])

    early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

    fitted_model = rnn2.fit(train_dataset, epochs=8,
                        validation_data=test_dataset, 
                        callbacks = [early_stop])

    rnn2.save(save_path + year + "_RNN")

Epoch 1/8


InvalidArgumentError: Graph execution error:

Detected at node 'sequential/embedding/embedding_lookup' defined at (most recent call last):
    File "c:\Users\danie\anaconda3\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\danie\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\danie\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\danie\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Users\danie\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 612, in start
      self.io_loop.start()
    File "c:\Users\danie\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\danie\anaconda3\lib\asyncio\base_events.py", line 570, in run_forever
      self